In [1]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.ndimage
import skimage
from skimage import io
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_ubyte

import cv2
from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma, CLAHE
)

import sklearn
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import telebot
import time
import tensorflow as tf
from tensorflow import keras

import lime
from lime import lime_image

import pandas as pd

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
  try:
    print("gpus existem")
    print(gpus)
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)
    
    
TELEBOT_TOKEN = "2058519653:AAG5Kf0Othtye8e13F5WPnBQQSdoCt47ifA"

bot = telebot.TeleBot("2058519653:AAG5Kf0Othtye8e13F5WPnBQQSdoCt47ifA")
bot.config['api_key'] = TELEBOT_TOKEN
bot.get_me()

gpus existem
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


{'ok': True,
 'result': {'id': 2058519653,
  'is_bot': True,
  'first_name': 'cnn_covid',
  'username': 'cnn_covid_bot',
  'can_join_groups': True,
  'can_read_all_group_messages': False,
  'supports_inline_queries': False}}

## Load Images

In [2]:
root_folder = "../../3_Images"

img_size = 300

images_train = []
labels_train = []

for idx, pathogen in enumerate(["Opacity", "COVID-19", "Normal"]):
  for img_filename in os.listdir(os.path.join(root_folder, "train", pathogen)):
    img = cv2.imread(os.path.join(root_folder, "train", pathogen, img_filename), 0)
    img = cv2.resize(img, (img_size, img_size), interpolation = cv2.INTER_CUBIC)
    img = skimage.img_as_float32(img)
    images_train.append(img)
    labels_train.append(idx)

images_test = []
labels_test = []
for idx, pathogen in enumerate(["Opacity", "COVID-19", "Normal"]):
  for img_filename in os.listdir(os.path.join(root_folder, "test", pathogen)):
    img = cv2.imread(os.path.join(root_folder, "test", pathogen, img_filename), 0)
    img = cv2.resize(img, (img_size, img_size), interpolation = cv2.INTER_CUBIC)
    img = skimage.img_as_float32(img)
    images_test.append(img)
    labels_test.append(idx)

print(len(images_train))
print(len(images_test))

6412
1545


## Cria Val, Train e Test

In [3]:
X_train = np.array(images_train).reshape((len(images_train), img_size, img_size))
Y_train = keras.utils.to_categorical(labels_train)
X_train, Y_train = sklearn.utils.shuffle(X_train, Y_train, random_state = 587)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 587)
X_val = np.stack((X_val,) * 3, axis = -1)

X_test = np.array(images_test).reshape((len(images_test), img_size, img_size))
X_test = np.stack((X_test,) * 3, axis = -1)
Y_test = keras.utils.to_categorical(labels_test)
X_test, Y_test = sklearn.utils.shuffle(X_test, Y_test)

print(np.unique(np.argmax(Y_train, axis = 1), return_counts = True))
print(np.unique(np.argmax(Y_val, axis = 1), return_counts = True))
print(np.unique(np.argmax(Y_test, axis = 1), return_counts = True))

(array([0, 1, 2]), array([1799, 1645, 1685]))
(array([0, 1, 2]), array([443, 414, 426]))
(array([0, 1, 2]), array([536, 502, 507]))


## Augmentation

In [4]:
np.random.seed = 587

class AugmentationSequence(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, augmentations):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.augment = augmentations

  def __len__(self):
    return int(np.ceil(len(self.x) / float(self.batch_size)))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
    
    aug_x = np.zeros(batch_x.shape)
    for idx in range(batch_x.shape[0]):
      aug = self.augment(image = batch_x[idx,:,:])
      aug_x[idx,:,:] = aug["image"]

    return np.stack((aug_x,) * 3, axis = -1), batch_y

augment = Compose([
  HorizontalFlip(),
  ShiftScaleRotate(shift_limit = 0.05, scale_limit = 0.05, rotate_limit = 15, border_mode = cv2.BORDER_CONSTANT),
  ElasticTransform(sigma = 20, alpha_affine = 20, border_mode = cv2.BORDER_CONSTANT),
  RandomBrightness(),
  RandomContrast(),
  RandomGamma()
])

batch_size = 16
train_generator = AugmentationSequence(X_train, Y_train, batch_size, augment)
steps_per_epoch = math.ceil(X_train.shape[0] / batch_size)

/home/dell/anaconda3/envs/tensorflow/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1744: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/dell/anaconda3/envs/tensorflow/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1770: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [5]:
steps_per_epoch

321

## Salvando datasets

In [6]:
import pickle
 

# Step 2
with open('input/train_dataset.pkl', 'wb') as file:
    pickle.dump(train_generator, file)

In [7]:
file_pi2 = open('input/train_dataset.pkl', 'rb') 
object_pi2 = pickle.load(file_pi2)


In [8]:
type(train_generator)

__main__.AugmentationSequence

In [9]:
type(object_pi2)

__main__.AugmentationSequence

In [10]:
np.save('input/X_test.npy', X_test)

In [11]:
np.save('input/Y_test.npy', Y_test)

In [12]:
np.save('input/X_val.npy', X_val)

In [13]:
np.save('input/Y_val.npy', Y_val)

In [14]:
np.save('input/X_train.npy', X_train)
np.save('input/Y_train.npy', Y_train)

In [15]:
data = np.load('input/X_test.npy')
# extract the first array

## Load Test 2

In [2]:
root_folder = "../../3_Images_Tests"

img_size = 300

images_train = []
labels_train = []

for idx, pathogen in enumerate(["Opacity", "COVID-19", "Normal"]):
  for img_filename in os.listdir(os.path.join(root_folder, "train", pathogen)):
    img = cv2.imread(os.path.join(root_folder, "train", pathogen, img_filename), 0)
    img = cv2.resize(img, (img_size, img_size), interpolation = cv2.INTER_CUBIC)
    img = skimage.img_as_float32(img)
    images_train.append(img)
    labels_train.append(idx)

images_test = []
labels_test = []
for idx, pathogen in enumerate(["Opacity", "COVID-19", "Normal"]):
  for img_filename in os.listdir(os.path.join(root_folder, "test", pathogen)):
    img = cv2.imread(os.path.join(root_folder, "test", pathogen, img_filename), 0)
    img = cv2.resize(img, (img_size, img_size), interpolation = cv2.INTER_CUBIC)
    img = skimage.img_as_float32(img)
    images_test.append(img)
    labels_test.append(idx)

print(len(images_train))
print(len(images_test))

5878
2394


In [3]:
X_train = np.array(images_train).reshape((len(images_train), img_size, img_size))
Y_train = keras.utils.to_categorical(labels_train)
X_train, Y_train = sklearn.utils.shuffle(X_train, Y_train, random_state = 587)

In [4]:
X_test = np.array(images_test).reshape((len(images_test), img_size, img_size))
X_test = np.stack((X_test,) * 3, axis = -1)
Y_test = keras.utils.to_categorical(labels_test)
X_test, Y_test = sklearn.utils.shuffle(X_test, Y_test)

## Salvando datasets 2

In [5]:
np.save('input/X_test2.npy', X_test)
np.save('input/Y_test2.npy', Y_test)
np.save('input/X_train2.npy', X_train)
np.save('input/Y_train2.npy', Y_train)